In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets 
import torchvision.transforms as transforms

In [2]:
input_size = 784        #Number of input neurons (image pixels)
hidden_size = 400       #Number of hidden neurons
out_size = 10           #Number of classes (0-9) 
epochs = 20             #How many times we pass our entire dataset into our network 
batch_size = 100        #Input size of the data during one iteration 
#batch_size=20
learning_rate = 0.001   #How fast we are learning
#learning_rate=0.01

In [3]:
train_dataset = datasets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

In [4]:

test_dataset = datasets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())


In [5]:

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [6]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()                    
        self.fc1 = nn.Linear(input_size, hidden_size)    #First Layer                           
        self.fc2 = nn.Linear(hidden_size, hidden_size)      #Second Layer Activation
        self.fc3 = nn.Linear(hidden_size, out_size)
        self.relu = nn.ReLU()
        self.init_weights()
        
    def init_weights(self):
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)

    def forward(self, x):                          
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out
    
#Create an object of the class, which represents our network 
net = Net(input_size, hidden_size, out_size)
CUDA = torch.cuda.is_available()
if CUDA:
    net = net.cuda()
#The loss function. The Cross Entropy loss comes along with Softmax. Therefore, no need to specify Softmax as well
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
#Train the network
for epoch in range(epochs):
    correct_train = 0
    running_loss = 0
    for i, (images, labels) in enumerate(train_loader):   
        #Flatten the image from size (batch,1,28,28) --> (100,1,28,28) where 1 represents the number of channels (grayscale-->1),
        # to size (100,784) and wrap it in a variable
        images = images.view(-1, 28*28)    
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()
            
        outputs = net(images)       
        _, predicted = torch.max(outputs.data, 1)                                              
        correct_train += (predicted == labels).sum() 
        loss = criterion(outputs, labels)                 # Difference between the actual and predicted (loss function)
        running_loss += loss.item()
        optimizer.zero_grad() 
        loss.backward()                                   # Backpropagation
        optimizer.step()                                  # Update the weights
        
    print('Epoch [{}/{}], Training Loss: {:.3f}, Training Accuracy: {:.3f}%'.format
          (epoch+1, epochs, running_loss/len(train_loader), (100*correct_train.double()/len(train_dataset))))
print("DONE TRAINING!")

with torch.no_grad():
    correct = 0
    for images, labels in test_loader:
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()
        images = images.view(-1, 28*28)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / len(test_dataset)))

Epoch [1/20], Training Loss: 0.239, Training Accuracy: 93.047%
Epoch [2/20], Training Loss: 0.087, Training Accuracy: 97.342%
Epoch [3/20], Training Loss: 0.056, Training Accuracy: 98.200%
Epoch [4/20], Training Loss: 0.039, Training Accuracy: 98.735%
Epoch [5/20], Training Loss: 0.027, Training Accuracy: 99.135%
Epoch [6/20], Training Loss: 0.024, Training Accuracy: 99.225%
Epoch [7/20], Training Loss: 0.019, Training Accuracy: 99.408%
Epoch [8/20], Training Loss: 0.019, Training Accuracy: 99.343%
Epoch [9/20], Training Loss: 0.013, Training Accuracy: 99.570%
Epoch [10/20], Training Loss: 0.012, Training Accuracy: 99.562%
Epoch [11/20], Training Loss: 0.013, Training Accuracy: 99.578%
Epoch [12/20], Training Loss: 0.012, Training Accuracy: 99.610%
Epoch [13/20], Training Loss: 0.010, Training Accuracy: 99.667%
Epoch [14/20], Training Loss: 0.012, Training Accuracy: 99.607%
Epoch [15/20], Training Loss: 0.009, Training Accuracy: 99.693%
Epoch [16/20], Training Loss: 0.007, Training Acc

In [7]:
#  lr = 0.001 
# batch_size = 20
#  optimiser = ADAM
# Accuracy : 97.89 % after 10 epochs

In [8]:
# batch_size = 100 
# optimiser = Adam
# Epoch [1/10], Training Loss: 0.235, Training Accuracy: 93.155%
# Epoch [2/10], Training Loss: 0.086, Training Accuracy: 97.315%
# Epoch [3/10], Training Loss: 0.056, Training Accuracy: 98.290%
# Epoch [4/10], Training Loss: 0.039, Training Accuracy: 98.740%
# Epoch [5/10], Training Loss: 0.031, Training Accuracy: 99.027%
# Epoch [6/10], Training Loss: 0.024, Training Accuracy: 99.177%
# Epoch [7/10], Training Loss: 0.019, Training Accuracy: 99.360%
# Epoch [8/10], Training Loss: 0.015, Training Accuracy: 99.502%
# Epoch [9/10], Training Loss: 0.015, Training Accuracy: 99.508%
# Epoch [10/10], Training Loss: 0.012, Training Accuracy: 99.570%
# DONE TRAINING!
# Accuracy of the network on the 10000 test images: 97.89 %

In [9]:
# lr = 0.01
#  batch_size = 20
#  optimiser = SGD
#  Accuracy : 97.37 % after 10 epochs